# 🔆 4-bit NormalFloat量化 🔆

- 我们有一组数据： \[2.3, -1.5, 0.7, -3.2, 1.8, -0.9, 0.0, 2.2\]
- 目标是将这些数据进行4-bit量化。

## 步骤一：数据标准化

首先，我们将数据标准化到 \([-1, 1]\) 区间，通常使用以下公式进行标准化：

$x' = \frac{2 \cdot (x - x_{\text{min}})}{x_{\text{max}} - x_{\text{min}}} - 1$

其中：

- $x$ 是原始数据。
- $x_{\text{min}}$ 是数据的最小值。
- $x_{\text{max}}$ 是数据的最大值。
- $x'$ 是标准化后的数据。

**(1) 计算数据的最小值和最大值**：

给定数据： \[2.3, -1.5, 0.7, -3.2, 1.8, -0.9, 0.0, 2.2\]

- 最小值 $x_{\text{min}} = -3.2$
- 最大值 $x_{\text{max}} = 2.3$


**(2) 应用标准化公式**：

将每个数据点 $x$ 应用公式：

$x' = \frac{2 \cdot (x - x_{\text{min}})}{x_{\text{max}} - x_{\text{min}}} - 1$

具体计算如下：

- 对于 $x = 2.3$ ：

  $
  x' = \frac{2 \cdot (2.3 - (-3.2))}{2.3 - (-3.2)} - 1 = \frac{2 \cdot 5.5}{5.5} - 1 = 2 - 1 = 1.0
  $​



**(3) 标准化后的数据：**

$
[1.0, -0.382, 0.418, -1.0, 0.818, -0.164, 0.164, 0.964]
$



In [1]:
x =  [2.3, -1.5, 0.7, -3.2, 1.8, -0.9, 0.0, 2.2]

x_min = min(x)
print("最小值：", x_min)

x_max = max(x)
print("最大值：", x_max)

x_new = [2*(item - x_min) / (x_max - x_min)-1 for item in x]

print("\n")
print("标准化后的数据样本：\n")
print(x_new)

最小值： -3.2
最大值： 2.3


标准化后的数据样本：

[1.0, -0.3818181818181817, 0.4181818181818182, -1.0, 0.8181818181818181, -0.16363636363636358, 0.1636363636363638, 0.9636363636363638]


## 步骤二：计算分位数

在本例中，我们将标准化数据到 $[-1, 1]$ 区间后，需要计算用于量化的分位数。

我们需要计算 $2^4 + 1 = 17$ 个分位数。

假设我们使用标准正态分布 $N(0,1)$ 的分位数来进行计算。具体的分位数计算如下（标准正态分布的分位数可以通过查表或函数计算得出）：

$Q_X \left( \frac{i}{17} \right)$

对于 $i = 1, 2, 3, \ldots, 17$ ，我们得到一组分位数。

In [2]:
import scipy.stats as stats

# 定义要计算的分位数
n = 17

percentiles = [i / (n+1) for i in range(1, n+1)]

# 分位数
print(percentiles)

[0.05555555555555555, 0.1111111111111111, 0.16666666666666666, 0.2222222222222222, 0.2777777777777778, 0.3333333333333333, 0.3888888888888889, 0.4444444444444444, 0.5, 0.5555555555555556, 0.6111111111111112, 0.6666666666666666, 0.7222222222222222, 0.7777777777777778, 0.8333333333333334, 0.8888888888888888, 0.9444444444444444]


In [3]:
# 计算标准正态分布的分位数
quantiles = stats.norm.ppf(percentiles)

print(quantiles)

[-1.59321882 -1.22064035 -0.96742157 -0.76470967 -0.5894558  -0.4307273
 -0.28221615 -0.1397103   0.          0.1397103   0.28221615  0.4307273
  0.5894558   0.76470967  0.96742157  1.22064035  1.59321882]


## 步骤三：计算量化值

接下来，我们使用公式计算每个量化值：

$
q_i = \frac{1}{2} \left( Q_X \left( \frac{i}{17} \right) + Q_X \left( \frac{i+1}{17} \right) \right)
$

我们将这些分位数带入公式中，计算每个 $q_i$ ：

例如，对于 $i = 0$ ：

$
q_0 = \frac{1}{2} \left( Q_X \left( 0 \right) + Q_X \left( \frac{1}{17} \right) \right) = \frac{1}{2} \left( -1.645 + -1.281 \right) = -1.463
$

重复这个步骤直到 \(i = 15\)，我们得到一组量化值：


In [4]:
# 计算量化值

quantized_values = []

for i in range(len(quantiles) - 1):
    q_i = 0.5 * (quantiles[i] + quantiles[i+1])
    quantized_values.append(q_i)

print(quantized_values)

[-1.4069295834352005, -1.0940309574745255, -0.8660656199440442, -0.6770827358180826, -0.510091548572618, -0.35647172317898285, -0.21096322297218512, -0.06985514944093106, 0.06985514944093106, 0.21096322297218517, 0.35647172317898285, 0.5100915485726178, 0.6770827358180826, 0.8660656199440442, 1.0940309574745255, 1.4069295834351998]


## 步骤四：数据映射到量化值

将标准化后的数据映射到最近的量化值：

In [5]:
import numpy as np

def get_new_value(quanti_values, st_v):
    stand_value = np.asarray(quanti_values)
    idx = (np.abs(stand_value - st_v)).argmin()
    return stand_value[idx]

In [6]:
# x_new 是标准化之后的数据

quan_data = [get_new_value(quantized_values, val) for val in x_new]

print(quan_data)

[1.0940309574745255, -0.35647172317898285, 0.35647172317898285, -1.0940309574745255, 0.8660656199440442, -0.21096322297218512, 0.21096322297218517, 0.8660656199440442]


## 对比三组数据

In [7]:
print("原始数据：\n", x)

原始数据：
 [2.3, -1.5, 0.7, -3.2, 1.8, -0.9, 0.0, 2.2]


In [8]:
print("原始数据在标准化之后的数据：\n", x_new)

原始数据在标准化之后的数据：
 [1.0, -0.3818181818181817, 0.4181818181818182, -1.0, 0.8181818181818181, -0.16363636363636358, 0.1636363636363638, 0.9636363636363638]


In [9]:
print("量化后的数据：\n", quan_data)

量化后的数据：
 [1.0940309574745255, -0.35647172317898285, 0.35647172317898285, -1.0940309574745255, 0.8660656199440442, -0.21096322297218512, 0.21096322297218517, 0.8660656199440442]



**为什么用 4-bit normalfloat 表示后，精度误差那么大？**  

- 标准化误差：将数据缩放到 [-1, 1] 区间，导致原始数据在数值上发生了变化。  
- 分位数误差：标准正态分布的分位数在 [-1, 1] 区间内的离散化导致数据进一步误差。  
- 低精度表示误差：4-bit 量化只能选择 16 个离散值之一，导致量化误差较大。  

## 4-Bit NormalFLoat 的优势

4-bit NormalFloat量化算法结合了分位数量化和标准化技术，具有以下主要优势：

1️⃣ **高效存储**
- **存储效率**：4-bit的表示方式大大减少了存储空间的需求。相比于传统的32-bit或16-bit浮点数，4-bit浮点数能显著节省存储空间，从而降低存储成本。
- **压缩比高**：通过量化，数据可以被压缩到原始大小的1/8，从而提高了存储和传输的效率。


  
**<font color=red>进一步讲解：</font>**  
32-bit和16-bit浮点数
- **32-bit浮点数**：每个数值占用32位（4字节）的存储空间。
- **16-bit浮点数**：每个数值占用16位（2字节）的存储空间。

- **4-bit浮点数**：每个数值仅占用4位（0.5字节）的存储空间。


📣 **比较4-bit浮点数与32-bit浮点数**
   - 32-bit浮点数占用4字节
   - 4-bit浮点数占用0.5字节

   压缩比计算：
   $
   \text{压缩比} = \frac{\text{4-bit浮点数的大小}}{\text{32-bit浮点数的大小}} = \frac{0.5}{4} = \frac{1}{8}
   $

因此，使用4-bit浮点数可以将数据压缩到原始大小的1/8。

📣 **比较4-bit浮点数与16-bit浮点数**
   - 16-bit浮点数占用2字节
   - 4-bit浮点数占用0.5字节

   压缩比计算：
   $
   \text{压缩比} = \frac{\text{4-bit浮点数的大小}}{\text{16-bit浮点数的大小}} = \frac{0.5}{2} = \frac{1}{4}
   $

   使用4-bit浮点数可以将数据压缩到原始大小的1/4。


**存储效率**：
   - 使用4-bit浮点数相比于32-bit浮点数，存储空间需求减少了8倍。
   - 使用4-bit浮点数相比于16-bit浮点数，存储空间需求减少了4倍。  

因此，4-bit浮点数在存储和传输数据时能够显著提高效率和降低成本，正是因为它极大地减少了存储空间需求和数据量。

2️⃣ **计算效率**
- **计算成本低**：在计算过程中，处理4-bit数据比处理高位数数据所需的计算资源更少。这意味着可以在更短的时间内完成计算任务。
- **加速模型推理**：在深度学习模型推理阶段，使用量化后的权重和激活值可以显著加速计算过程，特别是在资源有限的嵌入式设备和移动设备上。


4-bit NormalFloat量化算法通过结合分位数量化和标准化技术，在保持数据精度的同时，显著提高了存储和计算效率。这种方法特别适用于需要高效存储和快速计算的场景。

# 🔆 双重量化 🔆

使用8-bit浮点数对初始量化的结果进行进一步压缩，这一步的目标是通过减少位宽来进一步减少存储空间，同时尽量保持数据的准确性。

In [16]:
# 上一步的量化结果

print(quan_data)

[1.0940309574745255, -0.35647172317898285, 0.35647172317898285, -1.0940309574745255, 0.8660656199440442, -0.21096322297218512, 0.21096322297218517, 0.8660656199440442]


In [14]:
# ① 将数据分成块，计算每块的均值。例如，块大小为4

size = 4

# 分块
blocks = [quan_data[i : i+size] for i in range(0, len(quan_data), size)]

print("第1块：\n")
print(blocks[0])

print("\n")
print("第2块：\n")
print(blocks[1])

第1块：

[1.0940309574745255, -0.35647172317898285, 0.35647172317898285, -1.0940309574745255]


第2块：

[0.8660656199440442, -0.21096322297218512, 0.21096322297218517, 0.8660656199440442]


In [15]:
# 计算块均值
block_means = [np.mean(block) for block in blocks]

print(block_means)

[0.0, 0.4330328099720221]


In [18]:
# 将每块数据减去其均值
# 在初始量化的基础上进一步压缩，通过减去块均值来集中数据，使用更小的位宽进行存储。

second_quantized_data = []

for i , block in enumerate(blocks):
    mean = block_means[i]
    second_quantized_data.extend([val - mean for val in block])

print("二次量化后的数据：\n")
print(second_quantized_data)

二次量化后的数据：

[1.0940309574745255, -0.35647172317898285, 0.35647172317898285, -1.0940309574745255, 0.4330328099720221, -0.6439960329442072, -0.22206958699983692, 0.4330328099720221]
